# 機械学習フロー

1回目のSprintでは、機械学習の実践的な流れを抑えます。このSprintはWeek3,4の延長になり、引き続きKaggleの Home Credit Default Risk コンペティションを用います。特にここでは適切な 検証 を組み込むことを重視し、その上で前処理やモデルの選定を進めていきます。


適切な検証を行い、テストデータに対して高い汎化性能を持つモデルを完成させましょう。

## 【問題1】クロスバリデーション
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました（ホールドアウト法）。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。


具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。


sklearn.model_selection.KFold — scikit-learn 0.20.2 documentation

In [1]:
'''
ライブラリのインポート
'''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import missingno as msno
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder#LabelEncodingを行う（列数を増やさず、値を数字に変換する）
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import f1_score
%matplotlib inline
#pd.set_option('display.max_rows', 500)
#訓練、検証用データの読み込み
app_train = pd.read_csv("application_train.csv.zip")
app_test = pd.read_csv("application_test.csv")
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
#ラベルエンコーダオブジェクトを作成します
le = LabelEncoder()
le_count = 0

#列を反復処理します
for col in app_train:
    #値がオブジェクト型の時を条件指定
    if app_train[col].dtype == "object":
        # カテゴリ数が2つ以下の場合
        if len(list(app_train[col].unique())) <= 2:
            #訓練データで学習を行う
            le.fit(app_train[col])
            #訓練データ、検証データ共に変換する
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            #ラベルでエンコードされた列の数を追跡します
            le_count += 1
           
print("{}columnsをエンコードしました".format(le_count))

3columnsをエンコードしました


In [3]:
#ダミーデータ
app_train = pd.get_dummies(app_train)
print(app_train.shape)

app_test = pd.get_dummies(app_test)
print(app_test.shape)

(307511, 243)
(48744, 239)


In [4]:
#元データを残す
train0 = app_train.copy()
test0 = app_test.copy()

#ターゲット値
train_labels = app_train["TARGET"]
print(train_labels)
#id値
train_id = app_train["SK_ID_CURR"]
test_id = app_test["SK_ID_CURR"]


0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64


In [5]:
#A(app_train)とB(app_test)をjoin(両方含む)値に変換
train1, test1 = train0.align(test0, join = "inner", axis=1)

print("Training Features shape:", train1.shape)
print("Training Features shape:", test1.shape)

Training Features shape: (307511, 239)
Training Features shape: (48744, 239)


In [6]:
#特徴量リスト
features = list(train1.columns)
features

['SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NON

In [7]:
#欠損値を中央値で埋める
imputer = SimpleImputer(strategy = "median")
imputer.fit(train1)
train2 = imputer.transform(train1)
test2 = imputer.transform(test1)

In [8]:
#最小値0、最大値を１に正規化
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train2)
train3 = scaler.transform(train2)
test3 = scaler.transform(test2)

In [9]:
#訓練マスターデータ
df_train = pd.DataFrame(train3)
df_train.columns = features
#検証マスターデータ
df_test = pd.DataFrame(test3)
df_test.columns = features
df_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0.000000,0.0,0.0,1.0,0.0,0.001512,0.090287,0.090032,0.077441,0.256321,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.000003,0.0,0.0,0.0,0.0,0.002089,0.311736,0.132924,0.271605,0.045016,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000006,1.0,1.0,1.0,0.0,0.000358,0.022472,0.020025,0.023569,0.134897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000011,0.0,0.0,1.0,0.0,0.000935,0.066837,0.109477,0.063973,0.107023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000014,0.0,0.0,1.0,0.0,0.000819,0.116854,0.078975,0.117845,0.392880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,-0.000003,0.0,0.0,1.0,0.000000,0.000935,0.130787,0.073886,0.102132,0.257000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.000008,0.0,0.0,1.0,0.000000,0.000627,0.044387,0.061443,0.034792,0.491595,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000031,0.0,1.0,1.0,0.000000,0.001512,0.154373,0.265830,0.147026,0.260475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000073,0.0,0.0,1.0,0.105263,0.002474,0.382022,0.184872,0.382716,0.361433,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.000101,0.0,1.0,0.0,0.052632,0.001320,0.144944,0.118761,0.145903,0.134897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#ベースラインモデル
#df_train_x = df_train[["EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3", "DAYS_EMPLOYED"]]

df_train = df_train[:1000]
df_train_labels =train_labels[:1000]

#df_test = df_test[:5000]
nd_train = df_train.values
nd_test = df_test.values
nd_train_labels = df_train_labels.values
print(nd_train.shape)
#print(nd_test.shape)
print(type(nd_train))
#print(type(nd_test))
print(type(nd_train_labels))
#df_train1 = train_labels[:5000]
#X_train, X_test, y_train, y_test = train_test_split(df_train_x.values, df_train_y.values, test_size=0.2, random_state=0)
#print(f"特徴量の訓練データ = {X_train.shape} 特徴量の検証データ = {X_test.shape}")
#print(f"正解ラベルの訓練データ = {y_train.shape} 正解ラベルの検証データ = {y_test.shape}")

(1000, 239)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [12]:
#クロスバリデーション（交差検証）
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
n=1
kf.get_n_splits(nd_train)
print(kf)

#スコア値用の空リスト
roc_scores = []
auc_scores = []
#クロスバリデーションの実行
for train_index, test_index in kf.split(nd_train, nd_train_labels):
    print("{}回目----".format(n))
    print("TRAIN{}:, TEST:{}".format(len(train_index),len(test_index)))
    # kfoldで分割
    X_train, X_test = nd_train[train_index], nd_train[test_index]
    y_train, y_test = nd_train_labels[train_index], nd_train_labels[test_index]
    #ランダムフォレストでの学習
    rfc = RandomForestClassifier().fit(X_train, y_train)
    rfc_proba = rfc.predict_proba(X_test)[:,1]
    
    
    #AUCスコアの算出
    score = roc_auc_score(y_test, rfc_proba)
    auc_scores.append(score)
    
    print("AUC:",score)
    n += 1
    

KFold(n_splits=5, random_state=None, shuffle=False)
1回目----
TRAIN800:, TEST:200
AUC: 0.640735815602837
2回目----
TRAIN800:, TEST:200
AUC: 0.5144009216589862
3回目----
TRAIN800:, TEST:200
AUC: 0.6041441441441442
4回目----
TRAIN800:, TEST:200
AUC: 0.618469765528589
5回目----
TRAIN800:, TEST:200
AUC: 0.6676290760869565


In [13]:
print("各AUCスコア", auc_scores)
print("平均AUCスコア", np.mean(auc_scores))

各AUCスコア [0.640735815602837, 0.5144009216589862, 0.6041441441441442, 0.618469765528589, 0.6676290760869565]
平均AUCスコア 0.6090759446043026


## 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。


scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。


sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation


GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。



In [17]:
#ランダムフォレストのパラメータ
rfc_param = {RandomForestClassifier(): {"n_estimators":[i for i in range(2, 5)],
                                       "criterion":["gini", "entropy"],
                                       "max_depth":[i for i in range(1, 30)]
                                       }}
max_score = 0

#ランダムフォレストの実行
for model, param in tqdm(rfc_param.items()):
    clf = GridSearchCV(model, param, cv=5).fit(X_train, y_train)
    proba = clf.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, proba)
    
    if max_score < score:
        max_score = score
        best_param = clf.best_params_
        
print("best AUC:", max_score)
print("best params:", best_param)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.70s/it]

best AUC: 0.6423233695652173
best params: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 3}


## 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksからさまざまなアイデアを見つけ出して、列挙してください。

###### 解答
LightGBM
https://www.codexa.net/lightgbm-beginner/  
XGboost
https://toukei-lab.com/xgboost

## 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。


その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。



In [27]:
#GXBoost、ベースラインモデル
import xgboost as xgb
import category_encoders as ce

params = {'metric':'error',
         'objective':'binary:logistic',
         'n_estimators':50000,
         'booster':'gbtree',
         'learning_rate':0.01,
         'min_child_weight':1,
         'max_depth':5,
         'random_state':0,
         'colsample_bytree':1
         }
cls = xgb.XGBClassifier()
cls.set_params(**params)
cls.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)], eval_metric='error', verbose=1)


[23:34:56] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-error:0.08000
[1]	validation_0-error:0.08000
[2]	validation_0-error:0.08000
[3]	validation_0-error:0.08000
[4]	validation_0-error:0.08000
[5]	validation_0-error:0.08000
[6]	validation_0-error:0.08000
[7]	validation_0-error:0.08000
[8]	validation_0-error:0.08000
[9]	validation_0-error:0.08000
[10]	validation_0-error:0.08000
[11]	validation_0-error:0.08000
[12]	validation_0-error:0.08000
[13]	validation_0-error:0.08000
[14]	validation_0-error:0.08000
[15]	validation_0-error:0.08000
[16]	validation_0-error:0.08000
[17]	validation_0-error:0.08000
[18]	validation_0-error:0.08000
[19]	validation_0-error:0.08000
[20]	validation_0-error:0.08000
[21]	validat

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=5, metric='error',
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50000, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
xgb_proba = cls.predict_proba(X_test)[:,1]
xgb_score = roc_auc_score(y_test, xgb_proba)
print('AUC:', xgb_score)

AUC: 0.7783627717391304


In [42]:
#XGBoost、グリッドサーチ
cv_params = {'metric':['error'],
           'objective':['binary:logistic'],
           'n_estimators':[50000],
           'booster':['gbtree'],
           'learning_rate':[0.01],
           'min_child_weight':[1,5],
           'max_depth':[1,5],
           'random_state':[0],
           'colsample_bytree':[0.5,1.0],
           'subsample':[0.5,1.0]  
           }

#グリッドサーチの実行
xgb1 = xgb.XGBClassifier()
xgb1_grid = GridSearchCV(xgb1, cv_params, cv=KFold(5), scoring='accuracy')
xgb1_grid.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)], eval_metric='error', verbose=0)


C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:18:28] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:18:28] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:18:28] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:18:28] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[00:18:45] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:18:46] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:18:46] WARNING: ..\src\learner.cc:541: 
Parameters: { metric } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




C:\Users\ren30\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, m...
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=None,
                                     verbosity=None),
             param_grid={'booster': ['gbtree'], 'colsample_bytree': [0.5, 1.0],
                         'lea

In [52]:
#最も良かったパラメータを取得
print(xgb1_grid.best_params_)
print(xgb1_grid.best_score_)

{'booster': 'gbtree', 'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 1, 'metric': 'error', 'min_child_weight': 1, 'n_estimators': 50000, 'objective': 'binary:logistic', 'random_state': 0, 'subsample': 0.5}
0.9324999999999999


In [53]:
xgb1_proba = xgb1_grid.best_estimator_.predict_proba(X_test)[:,1]
xgb1_score = roc_auc_score(y_test, xgb1_proba)
print("AUC:",xgb1_score)

AUC: 0.4809782608695652


## 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

ランダムフォレストと、XGBootsとグリッドサーチで
平均AUCスコア 0.6090759446043026から
AUC: 0.7783627717391304まであがった